# Redes Sociais - APS 3

### Alunos: Arthur Barreto, Enricco Gemha e Felipe Catapano


Extraindo dados do maior marketplace de jogos da atualidade, *Steam* [$^1$](https://www.kaggle.com/datasets/nikdavis/steam-store-games), construímos uma rede de empresas desenvolvedoras de jogos que se conectam quando produzem jogos do mesmo subgênero, considerando mais de 27.033 jogos, de mais de 17.000 desenvolvedoras. Os vértices representam empresas desenvolvedoras de jogos e uma aresta não-direcionada indica um número de jogos a definir para cada subgênero que ambas produziram.

## Pré-requisitos

In [1]:
import graph_tool_extras as gte
import netpixi
from graph_tool import draw
import distribution as dst
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
PATH = 'decks.csv'

## Análise dos dados importados

Primeiro, abrimos e lemos o arquivo `steam.csv` para entendermos como os dados estão estruturados. 
Isso nos dá um *insight* de como realizar o *parsing* do arquivo e como extrair as informações que precisamos.
Com isso, definimos uma função `get_data` que realiza a subrotina de extração e limpeza de uma linha do arquivo `steam.csv`.

In [3]:
def get_data(line):
    # realiza o split por vírgula (.csv) e retorna a lista
    line_clean = line.split(',')
    return line_clean

Com isso, espiamos a saída da função `get_data` para entendermos como os dados estão estruturados.

In [4]:
with open(PATH) as file:

    # cria index de contagem para o loop abaixo.
    i = 0
    
    # ignora o cabeçalho.
    next(file)

    # Para não sobrecarregar este notebook
    # vamos espiar somente as 5 primeiras linhas.
    for line in file:

        # Chama função de leitura de dados.
        line_clean = get_data(line)
        
        # Para não sobrecarregar este notebook, vamos usar um contador
        # e um break para imprimir apenas as cinco primeiras linhas.
        i += 1
        if i == 5:
            break

Developer: Valve
Subgenres: ['FPS', 'Multiplayer']
Owners: 15000000.0

Developer: Valve
Subgenres: ['FPS', 'Multiplayer']
Owners: 7500000.0

Developer: Valve
Subgenres: ['FPS', 'World War II', 'Multiplayer']
Owners: 7500000.0

Developer: Valve
Subgenres: ['FPS', 'Multiplayer']
Owners: 7500000.0

Developer: Gearbox Software
Subgenres: ['FPS', 'Sci-fi']
Owners: 7500000.0



## Obtendo o valor de `fator_concorrencia`

Agora que possuímos as informações de `developer`, `owners` e `subgenres` de cada jogo, vamos calcular para cada `developer` o número de vendas (`owners`) somadas de todos os jogos que ela já produziu por `subgenre`. Isso nos ajudará a medir o impacto de cada `developer` em cada `subgenre` e, consequentemente, a concorrência entre `developers` em cada `subgenre`, escolhemos chamar isto de `fator_concorrencia`. 

## Criação do grafo

Utilizaremos a biblioteca [graph-tool](https://graph-tool.skewed.de/) somente para criação e visualização básica dos grafos, sem suporte de nenhum método ou função que não seja essencial.

In [4]:
g = gte.Graph(directed=False) # pois o grafo não é direcionado, como informado acima.

Antes, vamos definir duas funções auxiliares para facilitar a adição de novos nós e arestas, respectivamente.

In [5]:
def get_or_add_vertex(g, id):
    u = g.vertex_by_id(id)
    if u is None:
        u = g.add_vertex_by_id(id)
    return u

def get_or_add_edge(g, id1, id2):
    e = g.edge_by_ids(id1, id2)
    if e is None:
        e = g.add_edge_by_ids(id1, id2)
    return e

Antes, vamos definir duas funções auxiliares para facilitar a adição de novos nós e arestas, respectivamente.

In [7]:
with open(PATH) as file:

    # cria index de contagem para o loop abaixo.
    i = 1
    
    # ignora o cabeçalho.
    next(file)

    decklists = {}

    # Itera sobre as linhas do arquivo.
    for line in file:

        # Limpa os dados da linha.
        line_clean = get_data(line)

        # Remonta cada decklist.
        # A chave é o id do deck.
        # O valor é uma lista com os ids das cartas.
        # Tenta adquirir o valor da chave, caso não exista, cria uma lista vazia.
        decklist = decklists.get(line_clean[0], [])
        decklist.append(line_clean[1])
        decklists[line_clean[0]] = decklist

        # incrementa contador
        i += 1
        
# Itera sobre cada deck.
for deck in decklists.values():
    
    # adiciona os vértices
    for card in deck:
        get_or_add_vertex(g, card)

    # adiciona as arestas
    for j in range(len(deck)):
        for k in range(j+1, len(deck)):
            get_or_add_edge(g, deck[j], deck[k])

# Imprime a quantidade de linhas lidas.
print(f'Foram lidas {i} linhas.') 

# Imprime a quantidade de vértices.
print(f'Foram criados {g.num_vertices()} vértices.')

# Imprime a quantidade de arestas.
print(f'Foram criadas {g.num_edges()} arestas.')

Foram lidas 45477 linhas.


A seguir, devemos chamar `draw.sfdp_layout`, passando a rede, para rodar um algoritmo de posicionamento chamado SFDP [[1](#sfdp)].

Esse algoritmo usa uma ideia conhecida como [force-directed graph drawing](https://en.wikipedia.org/wiki/Force-directed_graph_drawing) para posicionar os vértices de forma a evidenciar agrupamentos.

In [8]:
layout = draw.sfdp_layout(g)

In [9]:
gte.move(g, layout)

## Armazenamento da rede

Para garantir a segurança da informação processada, devemos guardá-la em um arquivo na mesma pasta deste notebook.

In [10]:
g = gte.clean(g)
gte.save(g, 'spotify.net.gz')

## Visualização da rede

O próximo passo é a renderização da rede.

In [11]:
r = netpixi.render('spotify.net.gz', infinite=True)

Por fim, devemos ajustar a visualização da renderização.

In [12]:
r.vertex_default(size=4, bwidth=1)

In [13]:
r.edge_default(width=1)